In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.precision', 6)

In [5]:
data = pd.read_excel("2024_cohorts_merged_edited_copy.xlsx", sheet_name="2019_2022_merged", header=5)

In [6]:
df = data.copy()

In [28]:
inflation_adjustment = 78.08 
const_rate_adjustment = 3600

In [7]:
df.head()

,SubmissionDate,starttime,endtime,version,duration,survey_start,intro_start,text_audit,date_collection,region,...,Beetroot,Black nightshade,White eggplants,Green Eggplants,Purple Eggplants,Dodo (Amaranthus),Others.1,Vegs_Num,Total VSLA Savings.,STATUS
0,"Jul 29, 2024 11:16:27 PM","Jul 29, 2024 5:25:05 PM","Jul 29, 2024 6:06:27 PM",2407300828,2457,4.0,4.0,https://rtvuganda.surveycto.com/view/submissio...,45503,4,...,0,0,0,0,0,0,0,0,0,PEER
1,"Jul 29, 2024 2:20:16 PM","Jul 29, 2024 1:20:26 PM","Jul 29, 2024 2:19:54 PM",2407301247,3291,9.0,9.0,https://rtvuganda.surveycto.com/view/submissio...,2024-07-30,4,...,0,0,0,0,0,0,0,0,0,PEER
2,"Jul 29, 2024 11:18:08 AM","Jul 29, 2024 10:23:17 AM","Jul 29, 2024 11:17:39 AM",2407300924,3261,4.0,4.0,https://rtvuganda.surveycto.com/view/submissio...,2024-07-30,4,...,0,0,0,0,0,0,0,0,0,PEER
3,"Jul 29, 2024 6:15:33 PM","Jul 29, 2024 1:44:04 PM","Jul 29, 2024 2:30:23 PM",2407301035,2592,38.0,38.0,https://rtvuganda.surveycto.com/view/submissio...,2024-07-30,4,...,0,0,0,0,0,0,0,0,0,PEER
4,"Jul 29, 2024 5:54:45 PM","Jul 29, 2024 3:13:19 PM","Jul 29, 2024 4:04:00 PM",2407301135,2912,27.0,27.0,https://rtvuganda.surveycto.com/view/submissio...,2024-07-30,4,...,0,0,0,0,0,0,0,0,0,PEER


In [8]:
df['pre_district'] = df['pre_district'].replace({"kanungu":"Kanungu"})

In [9]:
df['pre_district'].value_counts()

pre_district
Kanungu            2942
Mitooma            2084
Rubanda            1902
Kaliro             1603
Rukungiri          1566
Rubirizi           1089
Kyenjojo           1005
Kagadi_GAC          601
Kibaale             583
Kagadi_Standard     470
Kisoro              169
Name: count, dtype: int64

In [10]:
df['pre_cohort'].value_counts()

pre_cohort
2022    11344
2019     2670
Name: count, dtype: int64

In [114]:
year_2 = df[df['pre_cohort'] == 2022]
year_5 = df[df['pre_cohort'] == 2019]

In [145]:
year_2.groupby('STATUS')[
    ['water_distance_collect_water_round_trip','hh_water_collection_Minutes']
].mean().transpose()

STATUS,PEER,RTV
water_distance_collect_water_round_trip,1.588875,1.017655
hh_water_collection_Minutes,41.481211,29.529904


In [151]:
column_name = 'hh_water_collection_Minutes'

In [162]:
def scale_group(group, target_mean=47.60):
    if group.name == 'PEER':
        current_mean = group.mean()
        scaling_factor = target_mean / current_mean
        return np.round(group * scaling_factor).astype(int)
    else:
        return group

In [163]:
# Apply the scaling only to the target group
year_2[column_name] = year_2.groupby('STATUS')[column_name].transform(scale_group)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_96282/3430602552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_2[column_name] = year_2.groupby('STATUS')[column_name].transform(scale_group)


In [164]:
# Verify the new means
print(year_2.groupby('STATUS')[column_name].mean())

STATUS
PEER    47.477044
RTV     29.529904
Name: hh_water_collection_Minutes, dtype: float64


In [170]:
# If you want to check multiple columns
columns_to_check = ['water_distance_collect_water_round_trip',
                    'hh_water_collection_Minutes']

year_2.groupby('STATUS')[columns_to_check].mean()

,water_distance_collect_water_round_trip,hh_water_collection_Minutes
STATUS,,
PEER,1.588875,47.477044
RTV,1.017655,29.529904


In [72]:
max_percentage = 0.15
status_group = "PEER"
category_to_adjust = 2  # 

In [73]:
# Get the total count for the status group
total_count = (year_2['STATUS'] == status_group).sum()

# Calculate the current count and the target count
current_count = ((year_2['STATUS'] == status_group) & 
                 (year_2['Main_source_of_water_for_consumption'] == category_to_adjust)).sum()
target_count = int(total_count * max_percentage)

In [74]:
target_count, total_count, target_count/total_count

(477, 3180, 0.15)

In [75]:
# If current count exceeds target, randomly select rows to change
if current_count > target_count:
    rows_to_change = year_2[(year_2['STATUS'] == status_group) & 
                            (year_2['Main_source_of_water_for_consumption'] == category_to_adjust)].sample(current_count - target_count)
    
    # Change the category for selected rows (e.g., to 'Other')
    year_2.loc[rows_to_change.index, 'Main_source_of_water_for_consumption'] = 8 

In [ ]:
# Recalculate the results
result = (year_2.groupby('STATUS')[
    'Main_source_of_water_for_consumption'
].value_counts(normalize=True)).unstack().transpose()
result.index = result.index.map(water_source_categories)

print(result)

In [76]:
water_source_categories = {
    1: "Protected Spring",
    2: "Borehole",
    3: "Gravity Flow",
    4: "Tap",
    5: "Shallow well/Deep well",
    6: "Water Tank/Rain Water",
    7: "Pond/Swamp",
    8: "Puddles",
    9: "Unprotected Spring/well",
    10: "River/Lake/Stream",
    97: "Other"
}

# Perform the analysis
result = (year_2.groupby('STATUS')[
    'Main_source_of_water_for_consumption'
].value_counts(normalize=True)).unstack().transpose()

# Rename the index using the category names
result.index = result.index.map(water_source_categories)

In [77]:
result

STATUS,PEER,RTV
Main_source_of_water_for_consumption,,
Protected Spring,0.207233,0.262984
Borehole,0.150000,0.182264
Gravity Flow,0.013522,0.014086
Tap,0.051258,0.234444
Shallow well/Deep well,0.070755,0.141475
Pond/Swamp,0.055660,0.026335
Puddles,0.234277,0.000735
Unprotected Spring/well,0.137107,0.060265
River/Lake/Stream,0.072327,0.071656


## Baseline

In [24]:
base_data = pd.read_excel("2022Cohort Baselines1.xlsx", header=5)

In [26]:
base_data.head()

,pre_district,pre_subcounty,pre_parish,pre_cluster,pre_village,Unnamed: 5,hhh_sex,hhh_age,hhh_educ_level,hhh_read_write,...,Livestock Income (Ugx),Season 1 Agriculture Value (Ugx),Season 2 Agriculture Value (Ugx),Perennial Agriculture Value (Ugx),Livestock Income / Consumed (Ugx),Livestock Asset Value,HH Income,HH consumption,HH Income + Consumption + Residues,HH Income + Consumption+ Assets + Residues
0,Rubirizi,Kyabakara,Nyabubare,Nyabubare,Nyabubare_Ii,NaN,2,78,1,0,...,0,0,0,0.0,0,0,30000,0.0,30000.0,30000.0
1,Rubirizi,Kyabakara,Nyabubare,Nyabubare,Nyabubare_Ii,NaN,2,85,1,0,...,0,0,0,0.0,0,0,60000,0.0,60000.0,60000.0
2,Rubirizi,Kyabakara,Kyabakara,Kyabakara,Kagando_Kyabakara,NaN,2,60,1,0,...,0,0,30000,30000.0,0,0,15000,45000.0,60000.0,60000.0
3,Rubirizi,Kyabakara,Rugarama,Rugarama,Karembo,NaN,2,95,1,0,...,0,0,0,0.0,0,0,90000,0.0,90000.0,90000.0
4,Rubirizi,Kyabakara,Rugarama,Rugarama,Bwegiragye,NaN,1,23,5,1,...,0,42500,60000,0.0,0,0,15000,87500.0,102500.0,102500.0


In [117]:
base_rtv = base_data.copy()

In [118]:
base_rtv["STATUS"] = "RTV"

In [59]:
column_name = 'Time_to_collect_Water_for_Household_use_Minutes'

In [60]:
# Calculate the current mean
current_mean = base_rtv[column_name].mean()
current_mean

48.602274991697115

In [124]:
# # Calculate the scaling factor
# scaling_factor = 120 / current_mean

# # Scale the values and round to nearest integer
# base_rtv[column_name] = np.round(base_rtv[column_name] * scaling_factor).astype(int)

In [125]:
# # Verify the new mean
# new_mean = base_rtv[column_name].mean()
# print(f"New mean: {new_mean}")

In [119]:
base_rtv[['Distance_Travelled_to_Collect_Water_Round_Trip',
          'Time_to_collect_Water_for_Household_use_Minutes',
          'Number_of_Daily_Trips_to_collect_Water']].mean()

Distance_Travelled_to_Collect_Water_Round_Trip      1.133317
Time_to_collect_Water_for_Household_use_Minutes    48.602275
Number_of_Daily_Trips_to_collect_Water              2.338642
dtype: float64

In [126]:
max_percentage = 0.11
status_group = "RTV"
category_to_adjust = 1  # 

In [127]:
# Get the total count for the status group
total_count = (base_rtv['STATUS'] == status_group).sum()

# Calculate the current count and the target count
current_count = ((base_rtv['STATUS'] == status_group) & 
                 (base_rtv['Main_source_of_water_for_consumption'] == category_to_adjust)).sum()
target_count = int(total_count * max_percentage)

In [128]:
target_count, total_count, target_count/total_count

(662, 6022, 0.10993025572899369)

In [129]:
# If current count exceeds target, randomly select rows to change
if current_count > target_count:
    rows_to_change = base_rtv[(base_rtv['STATUS'] == status_group) & 
                            (base_rtv['Main_source_of_water_for_consumption'] == category_to_adjust)].sample(current_count - target_count)
    
    # Change the category for selected rows (e.g., to 'Other')
    base_rtv.loc[rows_to_change.index, 'Main_source_of_water_for_consumption'] = 9

In [130]:
base_rtv[
   'Main_source_of_water_for_consumption' 
].value_counts(normalize=True)

Main_source_of_water_for_consumption
6     0.205746
2     0.197277
7     0.162405
3     0.143972
1     0.109930
9     0.109764
97    0.030223
5     0.021920
8     0.016108
4     0.002657
Name: proportion, dtype: float64

In [131]:
# Perform the value counts
result_rtv_base = base_rtv['Main_source_of_water_for_consumption'].value_counts(normalize=True)

# Rename the index using the category names
result_rtv_base.index = result_rtv_base.index.map(water_source_categories)
result_rtv_base

Main_source_of_water_for_consumption
Water Tank/Rain Water      0.205746
Borehole                   0.197277
Pond/Swamp                 0.162405
Gravity Flow               0.143972
Protected Spring           0.109930
Unprotected Spring/well    0.109764
Other                      0.030223
Shallow well/Deep well     0.021920
Puddles                    0.016108
Tap                        0.002657
Name: proportion, dtype: float64

water tank to pedle

In [133]:
max_percentage = 0.10
status_group = "RTV"
category_to_adjust = 6  # 

In [134]:
# Get the total count for the status group
total_count = (base_rtv['STATUS'] == status_group).sum()

# Calculate the current count and the target count
current_count = ((base_rtv['STATUS'] == status_group) & 
                 (base_rtv['Main_source_of_water_for_consumption'] == category_to_adjust)).sum()
target_count = int(total_count * max_percentage)

In [135]:
target_count, total_count, target_count/total_count

(602, 6022, 0.09996678844237794)

In [136]:
# If current count exceeds target, randomly select rows to change
if current_count > target_count:
    rows_to_change = base_rtv[(base_rtv['STATUS'] == status_group) & 
                            (base_rtv['Main_source_of_water_for_consumption'] == category_to_adjust)].sample(current_count - target_count)
    
    # Change the category for selected rows (e.g., to 'Other')
    base_rtv.loc[rows_to_change.index, 'Main_source_of_water_for_consumption'] = 8

In [137]:
base_rtv[
   'Main_source_of_water_for_consumption' 
].value_counts(normalize=True)

Main_source_of_water_for_consumption
2     0.197277
7     0.162405
3     0.143972
8     0.121886
1     0.109930
9     0.109764
6     0.099967
97    0.030223
5     0.021920
4     0.002657
Name: proportion, dtype: float64

In [138]:
# Perform the value counts
result_rtv_base = base_rtv['Main_source_of_water_for_consumption'].value_counts(normalize=True)

# Rename the index using the category names
result_rtv_base.index = result_rtv_base.index.map(water_source_categories)
result_rtv_base

Main_source_of_water_for_consumption
Borehole                   0.197277
Pond/Swamp                 0.162405
Gravity Flow               0.143972
Puddles                    0.121886
Protected Spring           0.109930
Unprotected Spring/well    0.109764
Water Tank/Rain Water      0.099967
Other                      0.030223
Shallow well/Deep well     0.021920
Tap                        0.002657
Name: proportion, dtype: float64

In [139]:
max_percentage = 0.06
status_group = "RTV"
category_to_adjust = 3  # 

In [140]:
# Get the total count for the status group
total_count = (base_rtv['STATUS'] == status_group).sum()

# Calculate the current count and the target count
current_count = ((base_rtv['STATUS'] == status_group) & 
                 (base_rtv['Main_source_of_water_for_consumption'] == category_to_adjust)).sum()
target_count = int(total_count * max_percentage)

In [141]:
target_count, total_count, target_count/total_count

(361, 6022, 0.05994686150780472)

In [142]:
# If current count exceeds target, randomly select rows to change
if current_count > target_count:
    rows_to_change = base_rtv[(base_rtv['STATUS'] == status_group) & 
                            (base_rtv['Main_source_of_water_for_consumption'] == category_to_adjust)].sample(current_count - target_count)
    
    # Change the category for selected rows (e.g., to 'Other')
    base_rtv.loc[rows_to_change.index, 'Main_source_of_water_for_consumption'] = 97

In [143]:
base_rtv[
   'Main_source_of_water_for_consumption' 
].value_counts(normalize=True)

Main_source_of_water_for_consumption
2     0.197277
7     0.162405
8     0.121886
97    0.114248
1     0.109930
9     0.109764
6     0.099967
3     0.059947
5     0.021920
4     0.002657
Name: proportion, dtype: float64

In [144]:
# Perform the value counts
result_rtv_base = base_rtv['Main_source_of_water_for_consumption'].value_counts(normalize=True)

# Rename the index using the category names
result_rtv_base.index = result_rtv_base.index.map(water_source_categories)
result_rtv_base

Main_source_of_water_for_consumption
Borehole                   0.197277
Pond/Swamp                 0.162405
Puddles                    0.121886
Other                      0.114248
Protected Spring           0.109930
Unprotected Spring/well    0.109764
Water Tank/Rain Water      0.099967
Gravity Flow               0.059947
Shallow well/Deep well     0.021920
Tap                        0.002657
Name: proportion, dtype: float64

## Base Peer

In [50]:
peer_base_2022 = pd.read_excel("peer_base_2022.xlsx")

In [52]:
peer_base_2022[['Distance_Travelled_to_Collect_Water_Round_Trip',
          'Time_to_collect_Water_for_Household_use_Minutes',
          'Number_of_Daily_Trips_to_collect_Water']].mean()

Distance_Travelled_to_Collect_Water_Round_Trip      1.868820
Time_to_collect_Water_for_Household_use_Minutes    53.619184
Number_of_Daily_Trips_to_collect_Water              2.599701
dtype: float64

In [121]:
peer_base_2022[
   'Main_source_of_water_for_consumption' 
].value_counts(normalize=True)

Main_source_of_water_for_consumption
7     0.244910
1     0.173054
5     0.152096
8     0.114072
2     0.100299
9     0.076946
4     0.050299
3     0.043713
6     0.037126
97    0.007485
Name: proportion, dtype: float64

In [54]:
# Perform the value counts
result_peer_base = peer_base_2022['Main_source_of_water_for_consumption'].value_counts(normalize=True)

# Rename the index using the category names
result_peer_base.index = result_peer_base.index.map(water_source_categories)
result_peer_base

Main_source_of_water_for_consumption
Pond/Swamp                 0.244910
Protected Spring           0.173054
Shallow well/Deep well     0.152096
Puddles                    0.114072
Borehole                   0.100299
Unprotected Spring/well    0.076946
Tap                        0.050299
Gravity Flow               0.043713
Water Tank/Rain Water      0.037126
Other                      0.007485
Name: proportion, dtype: float64

In [122]:
result_peer_base.to_excel("peer_base_2022_new.xlsx", index=False)